In [71]:
from imblearn.datasets import fetch_datasets
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.pipeline import make_pipeline
abalone_raw = fetch_datasets()['abalone']

In [72]:
abalone = pd.DataFrame(abalone_raw.data)

abalone['target'] = abalone_raw.target

abalone

,0,1,2,3,4,5,6,7,8,9,target
0,0.0,0.0,1.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,-1
1,0.0,0.0,1.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,1
2,1.0,0.0,0.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,-1
3,0.0,0.0,1.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,-1
4,0.0,1.0,0.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,1
...,...,...,...,...,...,...,...,...,...,...,...
4172,1.0,0.0,0.0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,-1
4173,0.0,0.0,1.0,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,-1
4174,0.0,0.0,1.0,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,-1
4175,1.0,0.0,0.0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,-1


In [73]:
abalone.loc[abalone['target'] == -1, 'target'] = 0

In [74]:
X_train, X_test, y_train, y_test = train_test_split(abalone.drop('target',axis=1), abalone['target'], test_size=0.33, random_state=42)

In [75]:
# Random Forest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector

num_pipe = SimpleImputer(strategy="mean", add_indicator=True)
cat_pipe = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
)

preprocessor_tree = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

rf_clf = make_pipeline(
    preprocessor_tree, RandomForestClassifier(n_estimators=300)
)

In [76]:
# Simple CV Baseline: RF
cv_result = cross_validate(rf_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [77]:
cv_result['test_score'].mean()

0.8301059529878486

In [78]:
## RANDOM PARAMETER GRID SEARCH
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(100, 1001, 100)]
# Number of features to consider at every split
max_features = ['log2','sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(5, 51, 5)]
max_depth.append(None)

criterion = ['gini', 'entropy']
# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.arange(2, 53, 5)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split':min_samples_split,
               'min_samples_leaf':min_samples_leaf,
               'bootstrap': bootstrap,
                'criterion':criterion}

print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_features': ['log2', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None], 'min_samples_split': [2, 7, 12, 17, 22, 27, 32, 37, 42, 47, 52], 'min_samples_leaf': [1], 'bootstrap': [False], 'criterion': ['gini', 'entropy']}


In [79]:
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                               param_distributions = random_grid,
                               scoring='roc_auc',
                                n_iter=1000,
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [80]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=1000,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 10, 15, 20, 25, 30, 35,
                                                      40, 45, 50, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1],
                                        'min_samples_split': [2, 7, 12, 17, 22,
                                                              27, 32, 37, 42,
                                                              47, 52],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
              

In [81]:
rf_random.best_score_

0.8643762306685687

In [82]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 47,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 5,
 'criterion': 'entropy',
 'bootstrap': False}

In [83]:
brf_random = RandomizedSearchCV(estimator = BalancedRandomForestClassifier(), 
                              param_distributions = random_grid,
                               scoring='roc_auc',
                                n_iter=1000,
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [84]:
brf_random.fit(X_train, y_train)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


RandomizedSearchCV(cv=3, estimator=BalancedRandomForestClassifier(),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'bootstrap': [False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 10, 15, 20, 25, 30, 35,
                                                      40, 45, 50, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1],
                                        'min_samples_split': [2, 7, 12, 17, 22,
                                                              27, 32, 37, 42,
                                                              47, 52],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
      

In [85]:
brf_random.best_score_

0.8647839279132489

In [86]:
brf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 12,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 5,
 'criterion': 'entropy',
 'bootstrap': False}

In [87]:
#Random Forest with oversampling using SMOTE and undersampling using RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

over_n = [float(x) for x in np.arange(0.2, 0.351, 0.05)]
# RandomUnderSampler undersampling
under_n = [float(x) for x in np.arange(0.35, 0.5, 0.05)]

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
               'randomforestclassifier__min_samples_split': min_samples_split,
               'randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'randomforestclassifier__bootstrap': bootstrap,
               'randomforestclassifier__criterion':criterion,
              'smote__sampling_strategy':over_n,
              'randomundersampler__sampling_strategy':under_n}

print(random_grid)

{'randomforestclassifier__n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'randomforestclassifier__max_features': ['log2', 'sqrt'], 'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None], 'randomforestclassifier__min_samples_split': [2, 7, 12, 17, 22, 27, 32, 37, 42, 47, 52], 'randomforestclassifier__min_samples_leaf': [1], 'randomforestclassifier__bootstrap': [False], 'randomforestclassifier__criterion': ['gini', 'entropy'], 'smote__sampling_strategy': [0.2, 0.25, 0.3, 0.35], 'randomundersampler__sampling_strategy': [0.35, 0.39999999999999997, 0.44999999999999996, 0.49999999999999994]}


In [88]:
# define pipeline
# oversample positive (minority) to be x% the number of negative (majority)
over = SMOTE(sampling_strategy = 0.1)
# randomly undersample negative (majority) to reduce the number of negative to x% of the positive (minority)
under = RandomUnderSampler(sampling_strategy = 0.25) 

pipeline = make_pipeline(over, under, RandomForestClassifier(n_estimators = 260,
 min_samples_split =2,
 min_samples_leaf=1,
 max_features= 'auto',
 max_depth= 90,
 bootstrap= False))

rf_sampling_random = RandomizedSearchCV(pipeline,
                               param_distributions = random_grid,
                               scoring='roc_auc',
                                n_iter=1000,
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [89]:
rf_sampling_random.fit(X_train,y_train)


Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('smote',
                                              SMOTE(sampling_strategy=0.1)),
                                             ('randomundersampler',
                                              RandomUnderSampler(sampling_strategy=0.25)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(bootstrap=False,
                                                                     max_depth=90,
                                                                     n_estimators=260))]),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'randomforestclassifier__bootstrap': [False],
                                        'randomfo...
                                        'randomforestclassifier__min_samples_split': [2,
                                                                                 

In [90]:
rf_sampling_random.best_score_


0.8689392512484106

In [91]:
rf_sampling_random.best_params_

{'smote__sampling_strategy': 0.2,
 'randomundersampler__sampling_strategy': 0.39999999999999997,
 'randomforestclassifier__n_estimators': 400,
 'randomforestclassifier__min_samples_split': 27,
 'randomforestclassifier__min_samples_leaf': 1,
 'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__max_depth': 5,
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__bootstrap': False}

In [160]:
# BALANCED BAGGING

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(300, 501, 100)]

# Method of selecting samples for training each tree
bootstrap = [True]

# max_features
max_features = [0.6,1.0]

#Sampling strategy
sampling_strategy = [0.2,0.3,0.4]

#sampler
sampler = [RandomUnderSampler(),SMOTE()]


# Create the random grid
bb_random_grid = {'n_estimators': n_estimators,
                 'bootstrap': bootstrap,
                 'sampling_strategy':sampling_strategy,
                 'sampler':sampler,
                 'max_features':max_features}


In [161]:
bb_random = RandomizedSearchCV(estimator = BalancedBaggingClassifier(base_estimator=HistGradientBoostingClassifier(random_state=42)), 
                               param_distributions = bb_random_grid,
                               scoring='roc_auc',
                               n_iter=15,
                               cv = 3, 
                               verbose=3, 
                               n_jobs = -1,
                               return_train_score = True)

In [162]:
bb_random.fit(X_train, y_train)


Fitting 3 folds for each of 15 candidates, totalling 45 fits


RandomizedSearchCV(cv=3,
                   estimator=BalancedBaggingClassifier(base_estimator=HistGradientBoostingClassifier(random_state=42)),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_features': [0.6, 1.0],
                                        'n_estimators': [300, 400, 500],
                                        'sampler': [RandomUnderSampler(),
                                                    SMOTE()],
                                        'sampling_strategy': [0.2, 0.3, 0.3,
                                                              0.4]},
                   return_train_score=True, scoring='roc_auc', verbose=3)

In [163]:
bb_random.best_score_

0.8544262294149597

In [164]:
bb_random.best_params_

{'sampling_strategy': 0.3,
 'sampler': RandomUnderSampler(),
 'n_estimators': 300,
 'max_features': 0.6,
 'bootstrap': True}

In [97]:
# LOGISTIC REGRESSION
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate



num_pipe = make_pipeline(
    StandardScaler()
)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_linear = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

lr_clf = make_pipeline(preprocessor_linear, LogisticRegression(max_iter=1000))
lr_clf.set_params(logisticregression__class_weight="balanced")

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=2,
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000024FABCAC1C8>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000024FAC83B948>)])),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [98]:
cv_result = cross_validate(lr_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [99]:
cv_result['test_score'].mean()

0.8437581473293818

In [113]:
# Support Vector Classification with RandomOverSampling
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline 

num_pipe = make_pipeline(
    MinMaxScaler(feature_range=(0, 1))
)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_svc = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

smote = SMOTE(sampling_strategy=0.1)

rus = RandomUnderSampler(sampling_strategy=0.2)

svc_clf = make_pipeline(preprocessor_svc, smote, rus, SVC(kernel='rbf',C=1))

svc_clf_no_sampling = make_pipeline(preprocessor_svc, SVC(kernel='rbf',C=1))

In [108]:
cv_result = cross_validate(svc_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [109]:
cv_result['test_score'].mean()

0.8228907866864945

In [114]:
parameters = {'smote__sampling_strategy':[float(x) for x in np.arange(0.2, 0.351, 0.05)],
              'randomundersampler__sampling_strategy':[float(x) for x in np.arange(0.35, 0.451, 0.05)],
              'svc__kernel':('linear', 'rbf', 'poly'), 
              'svc__C':[10,100,1000,5000,10000],
              'svc__gamma':[1,0.1,0.001,0.0001], 
              'svc__degree':[1,2,3]}

svc_grid = RandomizedSearchCV(svc_clf, param_distributions=parameters, n_iter=50, scoring="roc_auc", cv=3)


In [104]:
svc_grid.fit(X_train, y_train)


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=2,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('minmaxscaler',
                                                                                                MinMaxScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000024FAC4F9D48>),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))])..

In [105]:
svc_grid.best_score_

0.8620553360730187

In [106]:
svc_grid.best_params_

{'svc__kernel': 'rbf',
 'svc__gamma': 0.1,
 'svc__degree': 3,
 'svc__C': 5000,
 'smote__sampling_strategy': 0.2,
 'randomundersampler__sampling_strategy': 0.35}

In [119]:
# SVC WITHOUT SAMPLING
parameters = {'svc__kernel':('linear', 'rbf', 'poly'), 
              'svc__C':[10,100,1000,5000,10000],
              'svc__gamma':[1,0.1,0.001,0.0001], 
              'svc__degree':[1,2,3]}

svc_no_sampling_grid = RandomizedSearchCV(svc_clf_no_sampling, param_distributions=parameters, n_iter=50, scoring="roc_auc", cv=3, verbose=3)

In [120]:
cv_result = cross_validate(svc_clf_no_sampling, X_train, y_train, scoring="roc_auc", cv=5)

In [121]:
cv_result['test_score'].mean()

0.7543906022683047

In [122]:
svc_no_sampling_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=2,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('minmaxscaler',
                                                                                                MinMaxScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000024FB26B6088>),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),


In [124]:
svc_no_sampling_grid.best_score_

0.8591793969573618

In [125]:
svc_no_sampling_grid.best_params_


{'svc__kernel': 'rbf', 'svc__gamma': 0.001, 'svc__degree': 1, 'svc__C': 5000}